In [1]:
import math

import nacl.secret
import nacl.utils

from core import Sender, Receiver
from gptlm import GptLanguageModel

In [2]:
# Initialize the language model
# Restore the trained parameters of GPT-2-117M
lm = GptLanguageModel()
lm.enc.errors = 'strict'

INFO:tensorflow:Restoring parameters from ./external/gpt-2/models/117M/model.ckpt


In [3]:
# Cryptosystem configs
# Secret key encryption (based on https://pynacl.readthedocs.io/en/stable/secret/#example)
key = nacl.utils.random(nacl.secret.SecretBox.KEY_SIZE)
box = nacl.secret.SecretBox(key)

# Alice's secret message (padded with a witespace to make the ciphertext exactly 64 bytes long)
plaintext = b'cookie is in top drawer '

# Encrypt
ciphertext = box.encrypt(plaintext)
print('Plaintext:', plaintext)
print('Ciphertext:', ciphertext)

Plaintext: b'cookie is in top drawer '
Ciphertext: b'\xde;\xa4w@>S\x0b\xf4\x8aZ\xb3\xe1)\xdf\x85\xc3f+\xcc\x99\xcf\x84\x95\x10\x07\x9eI\x92\x16X\x8f\x0e_1\x13\xc29\x0f\xb6\xdf(\x9bJEz%H\x11\xbb!\x99\x933\xd2\xb3\xa1\x16\xf2\xc6f\xfa\xdc\xf3'


In [4]:
# Stegosystem configs
# Ciphertext length in bits. Set to 64 bytes
cipher_text_length = 64 * 8
# The imperceptibility parameter in patient-Huffman as introduced in the paper (https://arxiv.org/abs/1907.06679). 
# Smaller value means more imperceptible but longer stegotext.
tv_threshold = 0.1

alice_encoder = Sender(lm, None, None, cipher_text_length, tv_threshold, max_sequence_length=160, seed=123)

In [5]:
# Convert ciphertext into a list of bits in big-endian
ciphertext_in_bits = ''.join(['{:08b}'.format(x) for x in list(ciphertext)])
ciphertext_in_bits = [int(x) for x in ciphertext_in_bits]

In [6]:
# Generate the stegotext
stego_inds = alice_encoder.embed_bits(list(ciphertext_in_bits))
stego_text = lm.enc.decode(stego_inds)
print(stego_text)

A DUMBO: me :me

> DO not put off this player: Me: nobody else on his staff

>"anytime always gives me trouble"

"Pick me up and let's go"

"Moshe Hadid: don't dwell on someone else's wellbeing.

>"Heaven give me hope"

>"What am I going to do when I think that was all made up from dream songs and crap like that?" "Stone doesn't care. He stands above his regime era of genius." "James Carter: when people say anything is better, you've helped them over the line-up, you helped us down the road, though admittedly you're just as accomplished in some respects. " Kerrang: you're still able


In [7]:
# Bob decodes Alice's stegotext
bob_decoder = Receiver(lm, None, None, cipher_text_length, tv_threshold)
recovered_bits = bob_decoder.recover_bits(lm.enc.encode(stego_text), cipher_text_length)

In [8]:
# Check that Alice's ciphertext is correctly recovered by Bob
assert ''.join([str(x) for x in ciphertext_in_bits]) == ''.join([str(x) for x in recovered_bits])

In [9]:
# Convert the ciphertext back into bytes for decryption
base = [2**(7-n) for n in range(8)]
split = [recovered_bits[8 * i : 8 * i + 8] for i in range(len(recovered_bits) // 8)]
bytes([sum([base[i] * sp[i] for i in range(8)]) for sp in split])

b'\xde;\xa4w@>S\x0b\xf4\x8aZ\xb3\xe1)\xdf\x85\xc3f+\xcc\x99\xcf\x84\x95\x10\x07\x9eI\x92\x16X\x8f\x0e_1\x13\xc29\x0f\xb6\xdf(\x9bJEz%H\x11\xbb!\x99\x933\xd2\xb3\xa1\x16\xf2\xc6f\xfa\xdc\xf3'

In [10]:
# Decrypt
box.decrypt(_)

b'cookie is in top drawer '